# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression

from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


В распоряжении данные о пользователях «Мегалайн» пользующимися двуми тарифами «Смарт» или «Ультра». Они прописаны в столбце is_ultra: «Ультра» — 1, «Смарт» — 0.
-Предстоит работать с задачей классификации

## Разбейте данные на выборки

In [5]:
#Деление на три выборки
df_train, df_data = train_test_split(df, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df_data, test_size=0.5, random_state=12345)

In [6]:
print('Тренировочная выборка:', df_train.shape[0])
print('Валидационная выборка:', df_valid.shape[0])
print('Тестовая выборка:', df_test.shape[0])

Тренировочная выборка: 1928
Валидационная выборка: 643
Тестовая выборка: 643


In [7]:
#переменные для признаков и целевого признака тренировочной выборки
df_train_features = df_train.drop(['is_ultra'], axis=1)
df_train_target = df_train['is_ultra']

In [8]:
#переменные для признаков и целевого признака валидационной выборки
df_valid_features = df_valid.drop(['is_ultra'], axis=1)
df_valid_target = df_valid['is_ultra']

In [9]:
#переменные для признаков и целевого признака тестовой выборки
df_test_features = df_test.drop(['is_ultra'], axis=1)
df_test_target = df_test['is_ultra']

Данные разбиты на три выборки: тренировочная, валидационная и тестовая; в соотношении 60/20/20

Далее каждая выборка была разделена на целевой признак и обучающие признаки

## Исследуйте модели

**Модель дерево решений**

In [10]:
best_model = 0
best_result = 0

for depth in range(1, 11):
    
    model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_decision_tree.fit(df_train_features, df_train_target)
    
    predictions_valid_dt = model_decision_tree.predict(df_valid_features)
    
    result = accuracy_score(df_valid_target, predictions_valid_dt)
    
    print("max_depth =", depth, ": ", result)
    
    if result > best_result:
        
        # наилучшая модель
        best_model = depth 
        
        #  сохранение наилучшего значения метрики accuracy на валидационных данных
        best_result = result

print("Глубина дерева:", best_model, '=', best_result)

max_depth = 1 :  0.7356143079315708
max_depth = 2 :  0.7744945567651633
max_depth = 3 :  0.7791601866251944
max_depth = 4 :  0.7744945567651633
max_depth = 5 :  0.7838258164852255
max_depth = 6 :  0.776049766718507
max_depth = 7 :  0.7993779160186625
max_depth = 8 :  0.7931570762052877
max_depth = 9 :  0.7807153965785381
max_depth = 10 :  0.7884914463452566
Глубина дерева: 7 = 0.7993779160186625


In [11]:
model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=8, splitter='random')

model_decision_tree.fit(df_train_features, df_train_target)

predictions_valid_dt = model_decision_tree.predict(df_valid_features)

result = accuracy_score(df_valid_target, predictions_valid_dt)
    
print("Глубина дерева: =", result)

Глубина дерева: = 0.7962674961119751


'random' для выбора лучшего рандомного разбиения

In [12]:
model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=8, criterion='gini')

model_decision_tree.fit(df_train_features, df_train_target)

predictions_valid_dt = model_decision_tree.predict(df_valid_features)

result = accuracy_score(df_valid_target, predictions_valid_dt)
    
print("Глубина дерева: =", result)

Глубина дерева: = 0.7931570762052877


Критерий Джини, показывает сходства двух наборов данных. Обучаясь, дерево в каждом узле из возможных вопросов задаёт наилучший.

In [13]:
model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=8, splitter='best')

model_decision_tree.fit(df_train_features, df_train_target)

predictions_valid_dt = model_decision_tree.predict(df_valid_features)

result = accuracy_score(df_valid_target, predictions_valid_dt)
    
print("Глубина дерева: =", result)

Глубина дерева: = 0.7931570762052877


Стратегии 'best' для выбора лучшего разбиения

Изменение гиперпараметров для модели  дерево решений практически не влияют на исход, разница слишком мала

**Модель случайный лес**

In [14]:
best_model = 0
best_result = 0

for est in range(1, 11):
    
    #обучение модели с заданным количеством деревьев
    model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    
    # обучение модели на тренировочной выборке
    model_random_forest.fit(df_train_features, df_train_target)
    
    predictions_valid_rf = model_random_forest.predict(df_valid_features)
    
    # качество модели на валидационной выборке
    result = accuracy_score(df_valid_target, predictions_valid_rf)
    
    print("max_depth =", est, ": ", result)
    
    if result > best_result:
        # наилучшая модель
        best_model = est 
        #  сохранение наилучшего значения метрики accuracy на валидационных данных
        best_result = result 

#print("Accuracy наилучшей модели на валидационной выборке:", best_model, '=', best_result)
print("Количество деревьев:", best_model, '=', best_result)

max_depth = 1 :  0.7402799377916018
max_depth = 2 :  0.7589424572317263
max_depth = 3 :  0.7573872472783826
max_depth = 4 :  0.7729393468118196
max_depth = 5 :  0.7667185069984448
max_depth = 6 :  0.7791601866251944
max_depth = 7 :  0.7807153965785381
max_depth = 8 :  0.7869362363919129
max_depth = 9 :  0.7838258164852255
max_depth = 10 :  0.7807153965785381
Количество деревьев: 8 = 0.7869362363919129


In [15]:
    #обучение модели с заданным количеством деревьев
model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=best_model, criterion='entropy')
    
    # обучение модели на тренировочной выборке
model_random_forest.fit(df_train_features, df_train_target)
    
predictions_valid_rf = model_random_forest.predict(df_valid_features)
    
    # качество модели на валидационной выборке
result = accuracy_score(df_valid_target, predictions_valid_rf)
    
print("Accuracy наилучшей модели на валидационной выборке:", result)

Accuracy наилучшей модели на валидационной выборке: 0.7807153965785381


Для прироста информации воспользуемся критерием 'entropy'

In [16]:
    #обучение модели с заданным количеством деревьев
model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=best_model, criterion='gini')
    
    # обучение модели на тренировочной выборке
model_random_forest.fit(df_train_features, df_train_target)
    
predictions_valid_rf = model_random_forest.predict(df_valid_features)
    
    # качество модели на валидационной выборке
result = accuracy_score(df_valid_target, predictions_valid_rf)
    
print("Accuracy наилучшей модели на валидационной выборке:", result)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


Добавим критерий Джини для неоднородности 

Критерий Джини показывает себя лучше 'entropy'

Здесь аналогичная ситуация, влияние гиперпараметров слишком мало, его практичски нет

**Логистическая регрессия**

In [17]:
# инициализируем модель LinearRegression
model_lg = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
# обучаем модель на тренировочной выборке
model_lg.fit(df_train_features, df_train_target) 
# получаем предсказание модели на валидационной выборке
predictions_valid = model_lg.predict(df_valid_features) 

result = accuracy_score(df_valid_target, predictions_valid_rf)
print("Accuracy наилучшей модели на валидационной выборке:", result)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


Изменим значение параметра max_iter на 1000 

**ВЫВОД**
- **Модель дерево решений:** Глубина дерева 7: 0.7993779160186625
- **Модель случайный лес:** Количество деревьев 8: 0.7869362363919129
- **Логистическая регрессия:** Accuracy наилучшей модели: 0.7807153965785381

## Проверьте модель на тестовой выборке

In [18]:
predict_test1 = model_decision_tree.predict(df_test_features)
result = accuracy_score(df_test_target, predict_test1)

print("Accuracy наилучшей модели на тествой выборке, для дерева решений:", result)

predict_test2 = model_random_forest.predict(df_test_features)
result = accuracy_score(df_test_target, predict_test2)

print("Accuracy наилучшей модели на тествой выборке, для модели случайный лес:", result)

predict_test3 = model_lg.predict(df_test_features)
result = accuracy_score(df_test_target, predict_test3)

print("Accuracy наилучшей модели на тествой выборке, для логистической регресси:", result)

Accuracy наилучшей модели на тествой выборке, для дерева решений: 0.7791601866251944
Accuracy наилучшей модели на тествой выборке, для модели случайный лес: 0.7822706065318819
Accuracy наилучшей модели на тествой выборке, для логистической регресси: 0.7107309486780715


**ВЫВОД**
- На тестовой выборке лучше всего показала себя модель **случайного леса**, со значением 0.7822706065318819
- Хуже **логистическая регрессия** - 0.7107309486780715

Разница между вадидационной и тестовой выборками мала, это говорит о слабой обученности системы

## (бонус) Проверьте модели на адекватность

In [19]:
dummy_clf = DummyClassifier(strategy="most_frequent")

In [20]:
dummy_clf.fit(df_train_features, df_train_target)
dummy_clf.predict(df_valid_features)
dummy_clf.score(df_valid_features, df_valid_target)

0.6842923794712286

Получили значение равное 0.6842923794712286, что меньше Accuracy наилучшей модели, это говорит об адекватности получившейся модели.

**Общий вывод**

В распоряжении были данные о клиентах «Мегалайн» что перешли на один из двух тарифов.

Нужно было решить задачу классификации и построить модель с максимально большим значением accuracy. 

В процессе работы были:
- Разделены исходные данные на три выборки;
- Исследовано качество разных моделей;
- Проверено качество модели на тестовой выборке.

Наилучшее Accuracy у модели случайный лес: Количество деревьев 8: 0.7869362363919129

Разница между вадидационной и тестовой выборками мала, что говорит о слабой обученности системы